In [1]:
from ecodynelec.parameter import Parameter # Import the class to manipulate parameters
from ecodynelec.pipelines import execute
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from ecodynelec.updating import extract_entsoe_daily_generation_for_residuals

config = Parameter()
config.target= 'CH'
config.start= '2016-01-01 00:00'
config.end= '2021-12-31 23:59'
config.path.generation = "../etude_stat/test_data/downloads/generations/"
config.path.exchanges = "../etude_stat/test_data/downloads/exchanges/"
extract_entsoe_daily_generation_for_residuals(config, is_verbose=True)

Extracting days from Residual_model.xlsx
Computing CH generation data
Load generation data...
	Generation data.
Data loading: 57.04 sec.
Memory usage table: 1095.15 MB
Autocomplete...               1/1)...
Missing data identified: 358 (0.11%)
                                  CH
Hydro Pumped Storage               -
Hydro Run-of-river and poundage  240
Hydro Water Reservoir              -
Nuclear                            -
Solar                             47
Wind Onshore                      71
Extraction raw generation: 5.08 sec.             
	Extraction time: 64.55 sec.
	4/4 - Resample exchanges to D steps...
Saving CH generation data


{'CH':             Biomass_CH  Fossil_Brown_coal/Lignite_CH   
 2016-01-01         0.0                           0.0  \
 2016-01-02         0.0                           0.0   
 2016-01-03         0.0                           0.0   
 2016-01-04         0.0                           0.0   
 2016-01-05         0.0                           0.0   
 ...                ...                           ...   
 2021-12-27         0.0                           0.0   
 2021-12-28         0.0                           0.0   
 2021-12-29         0.0                           0.0   
 2021-12-30         0.0                           0.0   
 2021-12-31         0.0                           0.0   
 
             Fossil_Coal-derived_gas_CH  Fossil_Gas_CH  Fossil_Hard_coal_CH   
 2016-01-01                         0.0            0.0                  0.0  \
 2016-01-02                         0.0            0.0                  0.0   
 2016-01-03                         0.0            0.0                 

In [77]:
my_config = Parameter() # Initialize a configuration object
# Set base config
my_config.freq = "H"
my_config.ctry = ['AT','CH','DE','FR','IT', 'CZ']
my_config.data_cleaning = True
my_config.path.generation = "./test_data/downloads/generations/"
my_config.path.exchanges = "./test_data/downloads/exchanges/"

In [78]:
# Setup auto-downloading
my_config.server.useServer = False
my_config.server.username = 'aymericb5@gmail.com'
my_config.server.password = '8.U#Mf48nZn!eqc'

In [79]:
from ecodynelec.preprocessing.downloading import download
from ecodynelec.preprocessing.loading import import_generation, adjust_generation

p = my_config
ctry=['CH']
start='2016-01-01 00:00'
end='2021-12-31 23:59'
freq='D'
target=p.target
prod_gap=None
sg_data=None
path_gen=p.path.generation
path_imp=p.path.exchanges
savedir=p.path.savedir
net_exchange=p.net_exchanges
residual_global=p.residual_global
correct_imp=p.sg_imports
clean_data=p.data_cleaning
is_verbose=True

gen_preprocessed=None
n_hours=2
days_around=7
limit=.4

###########################
###### DOWNLOAD FROM SERVER
######
if p.server.useServer:
    if None in [p.path.generation, p.path.exchanges]: # If one path was not given
        raise KeyError("Can not download files: missing path raw_generation and/or raw_exchange to save files.")
    if is_verbose: print("Download Entso-E data from server...")
    (p.ctry, p.start, p.end, p.freq, p.target) = (ctry, start, end, freq, target)
    download(config=p, is_verbose=is_verbose) # Save files in a local dirrectory

###########################
###### GENERATE MIX
######
Gen = import_generation(path_gen=path_gen, path_prep=gen_preprocessed, ctry=ctry, start=start, end=end,
                        savedir=savedir, n_hours=n_hours, days_around=days_around, limit=limit,
                        clean_generation=False, is_verbose=is_verbose) # import generation data
Gen = adjust_generation(Gen, freq=freq, residual_global=residual_global, sg_data=sg_data,
                        prod_gap=prod_gap, is_verbose=is_verbose) # adjust the generation data

Load generation data...
	Generation data.
Data loading: 56.58 sec.
Memory usage table: 1095.15 MB
Autocomplete...               1/1)...
Missing data identified: 358 (0.11%)
                                  CH
Hydro Pumped Storage               -
Hydro Run-of-river and poundage  240
Hydro Water Reservoir              -
Nuclear                            -
Solar                             47
Wind Onshore                      71
Extraction raw generation: 2.11 sec.             
	Extraction time: 61.24 sec.
	4/4 - Resample exchanges to D steps...


In [80]:
Gen

{'CH':             Biomass_CH  Fossil_Brown_coal/Lignite_CH   
 2016-01-01         0.0                           0.0  \
 2016-01-02         0.0                           0.0   
 2016-01-03         0.0                           0.0   
 2016-01-04         0.0                           0.0   
 2016-01-05         0.0                           0.0   
 ...                ...                           ...   
 2021-12-27         0.0                           0.0   
 2021-12-28         0.0                           0.0   
 2021-12-29         0.0                           0.0   
 2021-12-30         0.0                           0.0   
 2021-12-31         0.0                           0.0   
 
             Fossil_Coal-derived_gas_CH  Fossil_Gas_CH  Fossil_Hard_coal_CH   
 2016-01-01                         0.0            0.0                  0.0  \
 2016-01-02                         0.0            0.0                  0.0   
 2016-01-03                         0.0            0.0                 

In [81]:
Gen['CH'].columns

Index(['Biomass_CH', 'Fossil_Brown_coal/Lignite_CH',
       'Fossil_Coal-derived_gas_CH', 'Fossil_Gas_CH', 'Fossil_Hard_coal_CH',
       'Fossil_Oil_CH', 'Fossil_Oil_shale_CH', 'Fossil_Peat_CH',
       'Geothermal_CH', 'Hydro_Pumped_Storage_CH',
       'Hydro_Run-of-river_and_poundage_CH', 'Hydro_Water_Reservoir_CH',
       'Marine_CH', 'Nuclear_CH', 'Other_fossil_CH', 'Other_renewable_CH',
       'Solar_CH', 'Waste_CH', 'Wind_Offshore_CH', 'Wind_Onshore_CH'],
      dtype='object')

In [82]:
residual_model = pd.read_excel('../support_files/Residual_model.xlsx', header=16, index_col=0)
#dates = residual_model.columns[0:36] # 36 days in 2015
#dates = residual_model.columns[36:36+36] # 36 days in 2016
dates = residual_model.columns
"""dates = ['1.21.2015','1.24.2015','1.25.2015','2.18.2015','2.21.2015','2.22.2015','3.15.2015','3.21.2015','3.22.2015','4.15.2015','4.18.2015','4.19.2015','5.20.2015','5.23.2015','5.24.2015','6.17.2015','6.20.2015','6.21.2015','7.15.2015','7.18.2015','7.19.2015','8.19.2015','8.22.2015','8.23.2015','9.16.2015','9.19.2015','9.20.2015','10.21.2015','10.24.2015','10.25.2015','11.18.2015','11.21.2015','11.22.2015','12.16.2015','12.19.2015','12.20.2015']"""
#dates = [pd.to_datetime(date, format='%m.%d.%Y') for date in dates]
dates = [date.strftime('%Y-%m-%d') for date in dates]
# I love copilot

In [83]:
csv_content = ''
csv_content += ';'
for date in dates:
    csv_content += date + ';'
csv_content += '\n'
for production in Gen['CH'].columns:
    csv_content += production.replace('_CH', '').replace('_', ' ') + ';'
    for date in dates:
        csv_content += str(Gen['CH'][production].loc[date]/1000).replace('.', ',') + ';'
    csv_content += '\n'

csv_file = open('2016-2021_entsoe_data_v2.csv', 'w')
csv_file.write(csv_content)
csv_file.close()

In [73]:
test = pd.read_excel('../support_files/Residual_model.xlsx')
test

,Days,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,OFEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2016-01-20 00:00:00,2016-01-23 00:00:00,2016-01-24 00:00:00,2016-02-17 00:00:00,2016-02-20 00:00:00,2016-02-21 00:00:00,2016-03-16 00:00:00,2016-03-19 00:00:00,2016-03-20 00:00:00,...,2021-09-19 00:00:00,2021-10-20 00:00:00,2021-10-23 00:00:00,2021-10-24 00:00:00,2021-11-17 00:00:00,2021-11-20 00:00:00,2021-11-21 00:00:00,2021-12-15 00:00:00,2021-12-18 00:00:00,2021-12-19 00:00:00
2,Centrales au fil de l’eau,30.4,25.1,23.7,32.4,29.1,31,29.9,24.9,24.3,...,47,27.3,25.6,24.3,26.7,22.7,22.4,31.4,25.2,24.2
3,Centrales à accumulation,84.1,36.7,26.4,67.5,30.3,21.1,69.8,27.9,26.7,...,23.6,51.9,40,22,80.9,46.3,34.5,92,34.3,31.2
4,Centrales nucléaires,71.1,70.9,71,71.2,71.2,70.6,71,70.9,71,...,41.1,41,41.3,41.4,41.3,41.4,41.4,71.3,71.2,71.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Centrales nucléaires,-1.315,-1.3643,-1.3756,-1.2794,-1.2626,-1.2292,-1.3239,-1.3241,-1.3686,...,-0.148837,-0.1904,-0.211812,-0.132425,-0.233925,-0.246613,-0.228563,-1.380075,-1.419213,-1.37025
62,Centrales therm. classiques et renouvelables,14.94894,14.60249,14.50105,15.8608,14.80327,13.77424,14.49163,13.86593,14.15694,...,15.743601,12.527453,9.464747,10.057681,12.783445,13.269401,13.247583,14.11565,11.55813,14.21507
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Expected week day,4,7,1,4,7,1,4,7,1,...,1,4,7,1,4,7,1,4,7,1


In [75]:
test.to_excel('test.xlsx')

In [ ]:
from ecodynelec.updating import update_residual_share
import os

path_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__file__))), 'support_files')
print('path is', path_dir)
update_residual_share(os.path.join(path_dir,"Residual_model.xlsx"), save=True)